In [ ]:
import pandas as pd

from rectools import Columns
from rectools.visuals import VisualApp, ItemToItemVisualApp

from pprint import pprint

import numpy as np
import pandas as pd

from tqdm.auto import tqdm

from implicit.nearest_neighbours import TFIDFRecommender, BM25Recommender
from implicit.als import AlternatingLeastSquares

from rectools import Columns
from rectools.dataset import Dataset
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
from rectools.models import ImplicitItemKNNWrapperModel, RandomModel, PopularModel
from rectools.model_selection import TimeRangeSplitter, cross_validate
from rectools.visuals import MetricsApp

In [ ]:
MODEL_DATA_DICT = [{'model': 'random',
  'i_split': 0,
  'prec@1': 0.0,
  'prec@10': 0.002666666666666667,
  'recall@10': 0.0008315950493672646,
  'novelty@10': 6.520986874262894,
  'serendipity@10': 0.0004220183486238532},
 {'model': 'popular',
  'i_split': 0,
  'prec@1': 0.05333333333333334,
  'prec@10': 0.023999999999999997,
  'recall@10': 0.03740973721177236,
  'novelty@10': 1.5807359233879539,
  'serendipity@10': 0.0001230437128980036},
 {'model': 'most_rated',
  'i_split': 0,
  'prec@1': 0.05333333333333334,
  'prec@10': 0.02666666666666667,
  'recall@10': 0.04225134477678418,
  'novelty@10': 1.5925431714409506,
  'serendipity@10': 0.00015146609102356583},
 {'model': 'tfidf_k=5',
  'i_split': 0,
  'prec@1': 0.05333333333333334,
  'prec@10': 0.021333333333333333,
  'recall@10': 0.023865993925579275,
  'novelty@10': 2.3611893254862615,
  'serendipity@10': 0.00046483180428134576},
 {'model': 'tfidf_k=10',
  'i_split': 0,
  'prec@1': 0.02666666666666667,
  'prec@10': 0.021333333333333336,
  'recall@10': 0.03992615769712139,
  'novelty@10': 2.1374512956864504,
  'serendipity@10': 0.00032667745997481555},
 {'model': 'bm25_k=10_k1=0.05_b=0.1',
  'i_split': 0,
  'prec@1': 0.02666666666666667,
  'prec@10': 0.02933333333333333,
  'recall@10': 0.04664484188165965,
  'novelty@10': 1.781881205653653,
  'serendipity@10': 0.0002709120345385858},
 {'model': 'random',
  'i_split': 1,
  'prec@1': 0.017543859649122806,
  'prec@10': 0.005263157894736843,
  'recall@10': 0.0007317526020904097,
  'novelty@10': 6.490783262148245,
  'serendipity@10': 0.0006049933251910133},
 {'model': 'popular',
  'i_split': 1,
  'prec@1': 0.05263157894736842,
  'prec@10': 0.05789473684210527,
  'recall@10': 0.015707029308143088,
  'novelty@10': 1.5884137454689444,
  'serendipity@10': 0.00018272881339885085},
 {'model': 'most_rated',
  'i_split': 1,
  'prec@1': 0.03508771929824561,
  'prec@10': 0.056140350877192984,
  'recall@10': 0.009918668187371998,
  'novelty@10': 1.6006279612022085,
  'serendipity@10': 0.00015479876160990716},
 {'model': 'tfidf_k=5',
  'i_split': 1,
  'prec@1': 0.05263157894736842,
  'prec@10': 0.05789473684210527,
  'recall@10': 0.048591489360456146,
  'novelty@10': 2.3261163377380436,
  'serendipity@10': 0.002616430444703231},
 {'model': 'tfidf_k=10',
  'i_split': 1,
  'prec@1': 0.05263157894736842,
  'prec@10': 0.05263157894736842,
  'recall@10': 0.010032924304988542,
  'novelty@10': 2.143504142765339,
  'serendipity@10': 0.0009174311926605506},
 {'model': 'bm25_k=10_k1=0.05_b=0.1',
  'i_split': 1,
  'prec@1': 0.07017543859649122,
  'prec@10': 0.05964912280701754,
  'recall@10': 0.010320823229416236,
  'novelty@10': 1.8094164875617524,
  'serendipity@10': 0.00034131470067505596},
 {'model': 'random',
  'i_split': 2,
  'prec@1': 0.015151515151515152,
  'prec@10': 0.0030303030303030303,
  'recall@10': 0.000876775470327413,
  'novelty@10': 6.336689727901374,
  'serendipity@10': 0.0005785049632863989},
 {'model': 'popular',
  'i_split': 2,
  'prec@1': 0.045454545454545456,
  'prec@10': 0.04242424242424242,
  'recall@10': 0.03998411752019977,
  'novelty@10': 1.6566376578061555,
  'serendipity@10': 0.0004501300103027036},
 {'model': 'most_rated',
  'i_split': 2,
  'prec@1': 0.045454545454545456,
  'prec@10': 0.03939393939393939,
  'recall@10': 0.024520628856449338,
  'novelty@10': 1.6681081982403094,
  'serendipity@10': 0.00033197599306611727},
 {'model': 'tfidf_k=5',
  'i_split': 2,
  'prec@1': 0.09090909090909091,
  'prec@10': 0.05,
  'recall@10': 0.03960590857136325,
  'novelty@10': 2.3789877971109377,
  'serendipity@10': 0.0015000245302457921},
 {'model': 'tfidf_k=10',
  'i_split': 2,
  'prec@1': 0.06060606060606061,
  'prec@10': 0.051515151515151514,
  'recall@10': 0.05353102602001291,
  'novelty@10': 2.2069212290765483,
  'serendipity@10': 0.0013463016566092655},
 {'model': 'bm25_k=10_k1=0.05_b=0.1',
  'i_split': 2,
  'prec@1': 0.09090909090909091,
  'prec@10': 0.039393939393939405,
  'recall@10': 0.038425713736665104,
  'novelty@10': 1.9013157591861718,
  'serendipity@10': 0.00039749219120508925}]

In [ ]:
META_DATA_DICT = {
    Columns.Model: ["random", "popular", "most_rated", "tfidf_k=5", "tfidf_k=10", "bm25_k=10_k1=0.05_b=0.1"],
    "k": [None, None, None, 5, 10, 10]
}

In [ ]:
model_data = pd.DataFrame(MODEL_DATA_DICT)
model_metadata = pd.DataFrame(META_DATA_DICT)

In [ ]:
metrics_app = MetricsApp.construct(
    models_metrics=model_data,
    models_metadata=model_metadata,
)

---

In [ ]:
interactions = pd.DataFrame({
    "user_id": [10, 10, 20, 20],
    "item_id": [1, 2, 4, 5],
})
interactions

In [ ]:
reco = pd.DataFrame({
    "user_id": [10, 10, 20, 20] + [10, 10, 20, 20] + [10, 10, 20, 20],
    "item_id": [12, 9, 10, 9] + [13, 14, 13, 14] + [15, 3, 7, 16],
    "rank": [1, 2, 1, 2] + [1, 2, 1, 2] + [1, 2, 1, 2],
    "model": ["Random model"] * 4 + ["Most rated"] * 4 + ["TF-IDF k=10"] * 4
})
reco

In [ ]:
item_data = pd.DataFrame({
    "item_id": [
        0, # Green book
        1, # Meir from Easttown
        2, # True detective
        3, # Oppenheimer
        4, # John Wick 3
        5, # The Dark Knight
        6, # The Lobster
        7, # Mad Max
        8, # Catch me if you can
        9, # Elf
        10, # Drive my car
        11, # Ice age
        12, # Submarine
        13, # The Shawshank Redemption
        14, # The Godfather
        15, # Se7en
        16, # Venom
    ],
    "name": [
        "Green Book", "Meir From Easttown", "True detective", "Oppenheimer", "John Wick 3", "The Dark Knight", "The Lobster", "Mad Max Fury Road", "Catch Me If You Can",
        "Elf", "Drive My Car", "Ice Age", "Submarine", "The Shawshank Redemption", "The Godfather", "Se7en", "Venom"
        ],
    "jenre": [
        "drama", "detective", "detective", "drama", "action", "action", "drama", "action", "action", 
        "comedy", "drama", "cartoon", "comedy", "drama", "drama", "detective", "action"
        ],
    "img_url": [
        'https://avatars.mds.yandex.net/get-kinopoisk-image/1900788/8e4206c9-fb99-4f43-9170-4586a5bc5b9b/3840x',
        'https://avatars.mds.yandex.net/get-kinopoisk-image/4774061/9c4e7a31-2b5a-4c8c-abc3-bfd67b8778cb/576x',
        'https://avatars.mds.yandex.net/get-kinopoisk-image/1946459/f5d887aa-dee4-4158-9713-61d41b04f94d/3840x',
        'https://avatars.mds.yandex.net/get-kinopoisk-image/6201401/9d064dee-0b29-4660-881a-1e7a3f81b3da/3840x',
        'https://avatars.mds.yandex.net/get-kinopoisk-image/1946459/bed1d2f9-cf3a-46a2-a6cd-cde4dc41ea43/3840x',
        'https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/4/4/7/3229744/446389851653464738.jpg',
        'https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/0/7/4/8164470/646328811653466467.jpg',
        'https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/9/7/1/8148179/927540511653466242.jpg',
        "https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/9/3/2/5239/743583001653464395.jpg",
        "https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/4/3/5/5534/550349591653464441.jpg",
        "https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/7/4/1/8365147/751798511651852864.jpg",
        "https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/3/5/9/4621953/35399f6271bf35ca324dc1d3beca10e0.jpeg",
        "https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/1/4/0/8111041/513084031653465751.jpg",
        "https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/1/7/5/7731571/48bab17014a55a36a3c050b1e166e2bf.jpg",
        "https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/2/7/1/3974172/7336e63fc5bed6c1e6022ea449f6cece.jpg",
        "https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/2/5/9/3977952/615146491653464847.jpg",
        "https://static.kinoafisha.info/k/movie_posters/1080x1920/upload/movie_posters/3/4/4/8329443/600773081653467108.jpg",
        ]
})
item_data

In [ ]:
visual_app = VisualApp.construct(
    reco=reco,
    interactions=interactions,
    item_data=item_data,
    selected_users={"detective user": 10, "action user": 20},  # users that we want to visualise
    formatters={"img_url": lambda x: f"<img src={x} width=100>"}  # process "img_url" links to html code
)